In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from datetime import datetime as dt

# Generation Data Cleaning

In [3]:
# Import datasets for power generation seperated by fuel types
# DONT RUN THIS - ACCIDENTALLY DELETED PORTIONS OF THE DATA USE 
df_2016 = pd.read_csv('gen_data/gen_by_fuel_2016.csv',parse_dates=True)
df_2017 = pd.read_csv('gen_data/gen_by_fuel_2016_2017.csv',parse_dates=True)
df_2018 = pd.read_csv('gen_data/gen_by_fuel_2017_2018.csv',parse_dates=True)
df_2019 = pd.read_csv('gen_data/gen_by_fuel_2018_2019.csv',parse_dates=True)
df_2020 = pd.read_csv('gen_data/gen_by_fuel_2019_2020.csv',parse_dates=True)
df_2021 = pd.read_csv('gen_data/gen_by_fuel_2020_2021.csv',parse_dates=True)

# Put datasets in list
df_gen_list = [df_2021,df_2020,df_2019,df_2018,df_2017,df_2016]

In [4]:
# Concatinate all of the dataframes together
df_gen = pd.concat(df_gen_list)

# Filter by columns we are interested in
df_gen_cut = df_gen[['datetime_beginning_ept','fuel_type','mw','is_renewable']]

In [5]:
# Filter by renewable energy
df_true = df_gen_cut[(df_gen_cut['is_renewable']==True)]

# Filter one last time by what we are interested in now that we filtered out fossil fuels
df_renew = df_true[['datetime_beginning_ept','fuel_type','mw']]

# Create pivot table of renewables based on the mw value
df = pd.pivot_table(df_renew, 
                    index = df_renew['datetime_beginning_ept'],
                    columns = df_renew['fuel_type'])

# Set index to datetime type
#df.index = pd.to_datetime(df.index)
# df.index.inferred_type == "datetime64" # check for datetime setting

In [6]:
df.to_csv('gen_data_full.csv')
df.index.names = [None]
df.columns = df.columns.droplevel()

# Weather Data Collection

### Chicago

In [7]:
# Precipitation Chicago
df_chi_2016p = pd.read_csv('precipitation_data/chicago/2016_2017_precip.csv')
df_chi_2018p = pd.read_csv('precipitation_data/chicago/2018_2019_precip.csv')
df_chi_2020p = pd.read_csv('precipitation_data/chicago/2020_2021_precip.csv')

df_chip = pd.concat([df_chi_2016p,df_chi_2018p,df_chi_2020p])

# Temperature Chicago
df_chi_2016t = pd.read_csv('temp_data/chicago/2016_2017_temp.csv')
df_chi_2018t = pd.read_csv('temp_data/chicago/2018_2019_temp.csv')
df_chi_2020t = pd.read_csv('temp_data/chicago/2020_2021_temp.csv')

df_chit = pd.concat([df_chi_2016t,df_chi_2018t,df_chi_2020t])

# Humidity Chicago
df_chi_2016h = pd.read_csv('humid_data/chicago/2016_2017_humid.csv')
df_chi_2018h = pd.read_csv('humid_data/chicago/2018_2019_humid.csv')
df_chi_2020h = pd.read_csv('humid_data/chicago/2020_2021_humid.csv')

df_chih = pd.concat([df_chi_2016h,df_chi_2018h,df_chi_2020h])

# Wind Chicago
df_chi_2016w = pd.read_csv('complete_wind/chicago/2016.csv')
df_chi_2017w = pd.read_csv('complete_wind/chicago/2017.csv')
df_chi_2018w = pd.read_csv('complete_wind/chicago/2018.csv')
df_chi_2019w = pd.read_csv('complete_wind/chicago/2019.csv')
df_chi_2020w = pd.read_csv('complete_wind/chicago/2020.csv')
df_chi_2021w = pd.read_csv('complete_wind/chicago/2021.csv')

df_chiw = pd.concat([df_chi_2016w,df_chi_2017w,df_chi_2018w,df_chi_2019w,df_chi_2020w,df_chi_2021w])

In [8]:
# Wind Speed Chicago
df_chiw['100_ws'] = np.sqrt((df_chiw['u100']**2)+(df_chiw['v100']**2))
df_chiw['10_ws'] = np.sqrt((df_chiw['u10']**2)+(df_chiw['v10']**2))
df_chiw['10n_ws'] = np.sqrt((df_chiw['u10n']**2)+(df_chiw['v10n']**2))

drop = ['u100','v100','u10n','u10','v10n','v10']
df_chiw = df_chiw.drop(columns = drop)

In [9]:
# Modify columns for easier analysis for ML with generation data
df_chip = df_chip.rename(columns = {'tp':'tp_chi'})
df_chit = df_chit.rename(columns = {'t':'t_chi'})
df_chih = df_chih.rename(columns = {'r':'r_chi'})
df_chiw = df_chiw.rename(columns = {'fg10':'fg10_chi',
                                   'i10fg':'i10fg_chi',
                                   '100_ws':'100_ws_chi',
                                   '10_ws':'10_ws_chi',
                                   '10n_ws':'10n_ws_chi'})

In [10]:
# Merge Chicago data together
df_chi = pd.merge(df_chip,df_chit, how = 'left',on = 'time')
df_chi = pd.merge(df_chi,df_chih, how = 'left',on = 'time')
df_chi = pd.merge(df_chi,df_chiw, how = 'left',on = 'time')

In [11]:
df_chi

,time,tp_chi,t_chi,r_chi,fg10_chi,i10fg_chi,100_ws_chi,10_ws_chi,10n_ws_chi
0,2016-01-01 00:00,0.0,269.98,71.09,12.07,11.92,8.676134,6.899804,7.165508
1,2016-01-01 01:00,0.0,269.62,72.89,12.01,11.90,9.059316,7.103196,7.375215
2,2016-01-01 02:00,0.0,269.23,73.14,11.90,11.01,9.064050,7.032283,7.298945
3,2016-01-01 03:00,0.0,268.96,72.78,11.01,10.36,8.906492,6.878408,7.156906
4,2016-01-01 04:00,0.0,268.97,72.15,10.44,10.37,9.161577,7.030512,7.308605
...,...,...,...,...,...,...,...,...,...
50396,2021-09-30 20:00,0.0,297.54,56.73,6.94,7.03,5.153882,4.250047,4.280000
50397,2021-09-30 21:00,0.0,297.70,57.41,7.17,7.14,5.692319,4.480279,4.400557
50398,2021-09-30 22:00,0.0,297.48,57.63,7.14,6.50,4.039715,3.159810,3.125540
50399,2021-09-30 23:00,0.0,297.25,63.14,6.50,5.02,4.029739,2.918784,2.801785


## Columbus

In [19]:
# Precipitation Columbus
df_col_2016p = pd.read_csv('precipitation_data/columbus/2016_2017_precip.csv')
df_col_2018p = pd.read_csv('precipitation_data/columbus/2018_2019_precip.csv')
df_col_2020p = pd.read_csv('precipitation_data/columbus/2020_2021_precip.csv')

df_colp = pd.concat([df_col_2016p,df_col_2018p,df_col_2020p])

# Temperature Columbus
df_col_2016t = pd.read_csv('temp_data/columbus/2016_2017_temp.csv')
df_col_2018t = pd.read_csv('temp_data/columbus/2018_2019_temp.csv')
df_col_2020t = pd.read_csv('temp_data/columbus/2020_2021_temp.csv')

df_colt = pd.concat([df_col_2016t,df_col_2018t,df_col_2020t])

# Humidity Columbus
df_col_2016h = pd.read_csv('humid_data/columbus/2016_2017_humid.csv')
df_col_2018h = pd.read_csv('humid_data/columbus/2018_2019_humid.csv')
df_col_2020h = pd.read_csv('humid_data/columbus/2020_2021_humid.csv')

df_colh = pd.concat([df_col_2016h,df_col_2018h,df_col_2020h])

# Wind Columbus
df_col_2016w = pd.read_csv('complete_wind/columbus/2016.csv')
df_col_2017w = pd.read_csv('complete_wind/columbus/2017.csv')
df_col_2018w = pd.read_csv('complete_wind/columbus/2018.csv')
df_col_2019w = pd.read_csv('complete_wind/columbus/2019.csv')
df_col_2020w = pd.read_csv('complete_wind/columbus/2020.csv')
df_col_2021w = pd.read_csv('complete_wind/columbus/2021.csv')

df_colw = pd.concat([df_col_2016w,df_col_2017w,df_col_2018w,df_col_2019w,df_col_2020w,df_col_2021w])

In [20]:
# Wind Speed Columbus
df_colw['100_ws'] = np.sqrt((df_colw['u100']**2)+(df_colw['v100']**2))
df_colw['10_ws'] = np.sqrt((df_colw['u10']**2)+(df_colw['v10']**2))
df_colw['10n_ws'] = np.sqrt((df_colw['u10n']**2)+(df_colw['v10n']**2))

df_colw = df_colw.drop(columns = drop)

In [21]:
# Modify columns for easier analysis for ML with generation data
df_colp = df_colp.rename(columns = {'tp':'tp_col'})
df_colt = df_colt.rename(columns = {'t':'t_col'})
df_colh = df_colh.rename(columns = {'r':'r_col'})
df_colw = df_colw.rename(columns = {'fg10':'fg10_col',
                                   'i10fg':'i10fg_col',
                                   '100_ws':'100_ws_col',
                                   '10_ws':'10_ws_col',
                                   '10n_ws':'10n_ws_col'})

In [22]:
# Merge Columbus data together
df_col = pd.merge(df_colp,df_colt, how = 'left', on = 'time')
df_col = pd.merge(df_col,df_colh, how = 'left', on = 'time')
df_col = pd.merge(df_col,df_colw, how = 'left',on = 'time')

## Pittsburgh

In [23]:
# Precipitation Pittsburgh
df_pitt_2016p = pd.read_csv('precipitation_data/pitt/2016_2017_precip.csv')
df_pitt_2018p = pd.read_csv('precipitation_data/pitt/2018_2019_precip.csv')
df_pitt_2020p = pd.read_csv('precipitation_data/pitt/2020_2021_precip.csv')

df_pittp = pd.concat([df_pitt_2016p,df_pitt_2018p,df_pitt_2020p])

# Temperature Pittsburgh
df_pitt_2016t = pd.read_csv('temp_data/pitt/2016_2017_temp.csv')
df_pitt_2018t = pd.read_csv('temp_data/pitt/2018_2019_temp.csv')
df_pitt_2020t = pd.read_csv('temp_data/pitt/2020_2021_temp.csv')

df_pittt = pd.concat([df_pitt_2016t,df_pitt_2018t,df_pitt_2020t])

# Humidity Pittsburgh
df_pitt_2016h = pd.read_csv('humid_data/pitt/2016_2017_humid.csv')
df_pitt_2018h = pd.read_csv('humid_data/pitt/2018_2019_humid.csv')
df_pitt_2020h = pd.read_csv('humid_data/pitt/2020_2021_humid.csv')

df_pitth = pd.concat([df_pitt_2016h,df_pitt_2018h,df_pitt_2020h])

# Wind Pittsburgh
df_pitt_2016w = pd.read_csv('complete_wind/pitt/2016.csv')
df_pitt_2017w = pd.read_csv('complete_wind/pitt/2017.csv')
df_pitt_2018w = pd.read_csv('complete_wind/pitt/2018.csv')
df_pitt_2019w = pd.read_csv('complete_wind/pitt/2019.csv')
df_pitt_2020w = pd.read_csv('complete_wind/pitt/2020.csv')
df_pitt_2021w = pd.read_csv('complete_wind/pitt/2021.csv')

df_pittw = pd.concat([df_pitt_2016w,df_pitt_2017w,df_pitt_2018w,df_pitt_2019w,df_pitt_2020w,df_pitt_2021w])

In [24]:
# Wind Speed Pittsburgh
df_pittw['100_ws'] = np.sqrt((df_pittw['u100']**2)+(df_pittw['v100']**2))
df_pittw['10_ws'] = np.sqrt((df_pittw['u10']**2)+(df_pittw['v10']**2))
df_pittw['10n_ws'] = np.sqrt((df_pittw['u10n']**2)+(df_pittw['v10n']**2))

df_pittw = df_pittw.drop(columns = drop)

In [25]:
# Modify columns for easier analysis for ML with generation data
df_pittp = df_pittp.rename(columns = {'tp':'tp_pitt'})
df_pittt = df_pittt.rename(columns = {'t':'t_pitt'})
df_pitth = df_pitth.rename(columns = {'r':'r_pitt'})
df_pittw = df_pittw.rename(columns = {'fg10':'fg10_pitt',
                                   'i10fg':'i10fg_pitt',
                                   '100_ws':'100_ws_pitt',
                                   '10_ws':'10_ws_pitt',
                                   '10n_ws':'10n_ws_pitt'})

In [26]:
# Merge Pittsburgh data together
df_pitt = pd.merge(df_pittp,df_pittt, how = 'left', on = 'time')
df_pitt = pd.merge(df_pitt,df_pitth, how = 'left', on = 'time')
df_pitt = pd.merge(df_pitt,df_pittw, how = 'left',on = 'time')

## DC

In [27]:
# Precipitation DC
df_dc_2016p = pd.read_csv('precipitation_data/dc/2016_2017_precip.csv')
df_dc_2018p = pd.read_csv('precipitation_data/dc/2018_2019_precip.csv')
df_dc_2020p = pd.read_csv('precipitation_data/dc/2020_2021_precip.csv')

df_dcp = pd.concat([df_dc_2016p,df_dc_2018p,df_dc_2020p])

# Temperature DC
df_dc_2016t = pd.read_csv('temp_data/dc/2016_2017_temp.csv')
df_dc_2018t = pd.read_csv('temp_data/dc/2018_2019_temp.csv')
df_dc_2020t = pd.read_csv('temp_data/dc/2020_2021_temp.csv')

df_dct = pd.concat([df_dc_2016t,df_dc_2018t,df_dc_2020t])

# Humidity DC
df_dc_2016h = pd.read_csv('humid_data/dc/2016_2017_humid.csv')
df_dc_2018h = pd.read_csv('humid_data/dc/2018_2019_humid.csv')
df_dc_2020h = pd.read_csv('humid_data/dc/2020_2021_humid.csv')

df_dch = pd.concat([df_dc_2016h,df_dc_2018h,df_dc_2020h])

# Wind DC
df_dc_2016w = pd.read_csv('complete_wind/dc/2016.csv')
df_dc_2017w = pd.read_csv('complete_wind/dc/2017.csv')
df_dc_2018w = pd.read_csv('complete_wind/dc/2018.csv')
df_dc_2019w = pd.read_csv('complete_wind/dc/2019.csv')
df_dc_2020w = pd.read_csv('complete_wind/dc/2020.csv')
df_dc_2021w = pd.read_csv('complete_wind/dc/2021.csv')

df_dcw = pd.concat([df_dc_2016w,df_dc_2017w,df_dc_2018w,df_dc_2019w,df_dc_2020w,df_dc_2021w])

In [28]:
# Wind Speed DC
df_dcw['100_ws'] = np.sqrt((df_dcw['u100']**2)+(df_dcw['v100']**2))
df_dcw['10_ws'] = np.sqrt((df_dcw['u10']**2)+(df_dcw['v10']**2))
df_dcw['10n_ws'] = np.sqrt((df_dcw['u10n']**2)+(df_dcw['v10n']**2))

df_dcw = df_dcw.drop(columns = drop)

In [29]:
# Modify columns for easier analysis for ML with generation data
df_dcp = df_dcp.rename(columns = {'tp':'tp_dc'})
df_dct = df_dct.rename(columns = {'t':'t_dc'})
df_dch = df_dch.rename(columns = {'r':'r_dc'})
df_dcw = df_dcw.rename(columns = {'fg10':'fg10_pitt',
                                   'i10fg':'i10fg_pitt',
                                   '100_ws':'100_ws_pitt',
                                   '10_ws':'10_ws_pitt',
                                   '10n_ws':'10n_ws_pitt'})

In [30]:
# Merge DC data together
df_dc = pd.merge(df_dcp,df_dct, how = 'left', on = 'time')
df_dc = pd.merge(df_dc,df_dch, how = 'left', on = 'time')
df_dc = pd.merge(df_dc,df_dcw, how = 'left', on = 'time')

## Philadelphia

In [31]:
# Precipitation Philadelphia
df_philly_2016p = pd.read_csv('precipitation_data/philly/2016_2017_precip.csv')
df_philly_2018p = pd.read_csv('precipitation_data/philly/2018_2019_precip.csv')
df_philly_2020p = pd.read_csv('precipitation_data/philly/2020_2021_precip.csv')

df_phillyp = pd.concat([df_philly_2016p,df_philly_2018p,df_philly_2020p])

# Temperature Philadelphia
df_philly_2016t = pd.read_csv('temp_data/philly/2016_2017_temp.csv')
df_philly_2018t = pd.read_csv('temp_data/philly/2018_2019_temp.csv')
df_philly_2020t = pd.read_csv('temp_data/philly/2020_2021_temp.csv')

df_phillyt = pd.concat([df_philly_2016t,df_philly_2018t,df_philly_2020t])

# Humidity Philadelphia
df_philly_2016h = pd.read_csv('humid_data/philly/2016_2017_humid.csv')
df_philly_2018h = pd.read_csv('humid_data/philly/2018_2019_humid.csv')
df_philly_2020h = pd.read_csv('humid_data/philly/2020_2021_humid.csv')

df_phillyh = pd.concat([df_philly_2016h,df_philly_2018h,df_philly_2020h])

# Wind Philadelphia
df_philly_2016w = pd.read_csv('complete_wind/philly/2016.csv')
df_philly_2017w = pd.read_csv('complete_wind/philly/2017.csv')
df_philly_2018w = pd.read_csv('complete_wind/philly/2018.csv')
df_philly_2019w = pd.read_csv('complete_wind/philly/2019.csv')
df_philly_2020w = pd.read_csv('complete_wind/philly/2020.csv')
df_philly_2021w = pd.read_csv('complete_wind/philly/2021.csv')

df_phillyw = pd.concat([df_philly_2016w,df_philly_2017w,df_philly_2018w,df_philly_2019w,df_philly_2020w,df_philly_2021w])

In [32]:
# Wind Speed Philadelphia
df_phillyw['100_ws'] = np.sqrt((df_phillyw['u100']**2)+(df_phillyw['v100']**2))
df_phillyw['10_ws'] = np.sqrt((df_phillyw['u10']**2)+(df_phillyw['v10']**2))
df_phillyw['10n_ws'] = np.sqrt((df_phillyw['u10n']**2)+(df_phillyw['v10n']**2))

df_phillyw = df_phillyw.drop(columns = drop)

In [33]:
# Modify columns for easier analysis for ML with generation data
df_phillyp = df_phillyp.rename(columns = {'tp':'tp_philly'})
df_phillyt = df_phillyt.rename(columns = {'t':'t_philly'})
df_phillyh = df_phillyh.rename(columns = {'r':'r_philly'})
df_phillyw = df_phillyw.rename(columns = {'fg10':'fg10_philly',
                                   'i10fg':'i10fg_philly',
                                   '100_ws':'100_ws_philly',
                                   '10_ws':'10_ws_philly',
                                   '10n_ws':'10n_ws_philly'})

In [34]:
# Merge Philadelphia data together
df_philly = pd.merge(df_phillyp,df_phillyt, how = 'left', on = 'time')
df_philly = pd.merge(df_philly,df_phillyh, how = 'left', on = 'time')
df_philly = pd.merge(df_philly,df_phillyw, how = 'left', on = 'time')

## All together

In [35]:
# Merge all weather data together
df_w = pd.merge(df_chi,df_col,how = 'left', on = 'time')
df_w= pd.merge(df_w,df_pitt,how = 'left', on = 'time')
df_w = pd.merge(df_w,df_dc,how = 'left', on = 'time')
df_weather = pd.merge(df_w,df_philly,how = 'left', on = 'time')

In [36]:
# Set time index
df_weather = df_weather.set_index('time')

In [37]:
df_weather.to_csv('weather_data_full.csv')

In [39]:
df_weather

,tp_chi,t_chi,r_chi,fg10_chi,i10fg_chi,100_ws_chi,10_ws_chi,10n_ws_chi,tp_col,t_col,...,10_ws_pitt_y,10n_ws_pitt_y,tp_philly,t_philly,r_philly,fg10_philly,i10fg_philly,100_ws_philly,10_ws_philly,10n_ws_philly
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00,0.0,269.98,71.09,12.07,11.92,8.676134,6.899804,7.165508,0.0,272.07,...,2.153927,2.074078,0.0,278.40,60.07,6.60,6.28,5.824577,3.534204,3.512165
2016-01-01 01:00,0.0,269.62,72.89,12.01,11.90,9.059316,7.103196,7.375215,0.0,272.12,...,2.324586,2.284841,0.0,277.91,60.44,6.28,5.85,5.672257,3.314363,3.270015
2016-01-01 02:00,0.0,269.23,73.14,11.90,11.01,9.064050,7.032283,7.298945,0.0,272.11,...,2.411079,2.382100,0.0,277.44,60.82,5.85,5.60,5.544231,3.119695,3.052769
2016-01-01 03:00,0.0,268.96,72.78,11.01,10.36,8.906492,6.878408,7.156906,0.0,272.14,...,2.744668,2.744668,0.0,277.05,60.99,5.63,5.57,5.313116,2.983186,2.916110
2016-01-01 04:00,0.0,268.97,72.15,10.44,10.37,9.161577,7.030512,7.308605,0.0,272.15,...,2.781726,2.759366,0.0,276.75,60.98,5.64,5.66,5.333610,3.045997,3.001283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 20:00,0.0,297.54,56.73,6.94,7.03,5.153882,4.250047,4.280000,0.0,295.09,...,2.578410,2.698240,0.0,290.53,48.13,7.83,7.21,5.004498,3.500357,3.563495
2021-09-30 21:00,0.0,297.70,57.41,7.17,7.14,5.692319,4.480279,4.400557,0.0,295.25,...,2.238772,2.319310,0.0,290.31,49.10,7.21,6.11,4.562127,3.026896,3.068061
2021-09-30 22:00,0.0,297.48,57.63,7.14,6.50,4.039715,3.159810,3.125540,0.0,295.46,...,2.045336,2.035706,0.0,290.32,50.06,6.11,4.70,4.156501,2.571245,2.571245


# Average Weather Data

In [70]:
dfw_avg = pd.DataFrame()

dfw_avg['tp'] = df_weather[['tp_chi', 'tp_col','tp_pitt','tp_dc','tp_philly']].mean(axis=1)
dfw_avg['t'] = df_weather[['t_chi', 't_col','t_pitt','t_dc','t_philly']].mean(axis=1)
dfw_avg['r'] = df_weather[['r_chi', 'r_col','r_pitt','r_dc','r_philly']].mean(axis=1)
dfw_avg['i10fg'] = df_weather[['i10fg_chi', 'i10fg_col','i10fg_pitt','i10fg_dc','i10fg_philly']].mean(axis=1)
dfw_avg['fg10'] = df_weather[['fg10_chi', 'fg10_col','fg10_pitt','fg10_dc','fg10_philly']].mean(axis=1)
dfw_avg['100_ws'] = df_weather[['100_ws_chi', '100_ws_col','100_ws_pitt','100_ws_dc','100_ws_philly']].mean(axis=1)
dfw_avg['10_ws'] = df_weather[['10_ws_chi', '10_ws_col','10_ws_pitt','10_ws_dc','10_ws_philly']].mean(axis=1)
dfw_avg['10n_ws'] = df_weather[['10n_ws_chi', '10n_ws_col','10n_ws_pitt','10n_ws_dc','10n_ws_philly']].mean(axis=1)
dfw_avg

,tp,t,r,i10fg,fg10,100_ws,10_ws,10n_ws
time,,,,,,,,
2016-01-01 00:00,0.0,275.152,65.364,7.598,7.858,5.694857,3.825055,3.867771
2016-01-01 01:00,0.0,274.814,65.654,7.476,7.704,5.847745,3.902233,3.947746
2016-01-01 02:00,0.0,274.478,65.590,7.306,7.604,5.966656,3.920689,3.959180
2016-01-01 03:00,0.0,274.208,65.206,7.260,7.444,6.062850,4.005319,4.049599
2016-01-01 04:00,0.0,274.016,64.910,7.504,7.460,6.220294,4.089588,4.132097
...,...,...,...,...,...,...,...,...
2021-09-30 20:00,0.0,294.048,46.522,6.404,6.626,3.984230,2.960278,3.064865
2021-09-30 21:00,0.0,294.098,47.426,5.884,6.440,4.066085,2.896726,2.948296
2021-09-30 22:00,0.0,293.736,50.760,4.200,5.884,3.449292,2.218185,2.206734


In [71]:
dfw_avg.to_csv('weather_data_avg.csv')